# Import Libraries

In [17]:
import pandas as pd
import pickle as pkl
import time
import regex as re
from collections import Counter
import numpy as np
import matplotlib.pylab as plt

# Read Files

In [18]:
dataset = pd.read_csv("data/ratings_books.csv", low_memory=False).fillna("")
dataset.columns = ['User-ID', 'ISBN', 'Book-Rating']

# Statistics

In [19]:
print("Original dataset statistics: ")
print(f"> No. of users: {len(dataset['User-ID'].unique())}")
print(f"> No. of Books: {len(dataset['ISBN'].unique())}")
print(f"> No. of Interaction: {dataset.shape[0]}")

Original dataset statistics: 
> No. of users: 105283
> No. of Books: 340556
> No. of Interaction: 1149780


# Filter

In [20]:
def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

In [21]:
dataset = filter_rows_by_values(dataset, "Book-Rating", [0])

print(dataset['Book-Rating'].value_counts())

# statistics on explicit dataset
print("Explicit dataset statistics: ")
print(f"> No. of users: {len(dataset['User-ID'].unique())}")
print(f"> No. of Books: {len(dataset['ISBN'].unique())}")
print(f"> No. of Interaction: {dataset.shape[0]}")

8     103736
10     78610
7      76457
9      67541
5      50974
6      36924
4       8904
3       5996
2       2759
1       1770
Name: Book-Rating, dtype: int64
Explicit dataset statistics: 
> No. of users: 77805
> No. of Books: 185973
> No. of Interaction: 433671


In [22]:
# To check if there is any user with more than toprate interaction in the preprocessed dataset
# The correct output will be zero
uid_value_counts = dataset['User-ID'].value_counts()
print("The number of users with more than 200 interactions:", len(uid_value_counts[uid_value_counts > 200]))

The number of users with more than 200 interactions: 144


In [23]:
# To remove the users with fewer than 5 interaction we first count the number of interactino per user and add a new column (`Count`) in the dataframe.
# This column shows the number of interaction per user in the dataset
users_counts = dataset['User-ID'].value_counts()
users_counts = users_counts.to_dict() #converts to dictionary
dataset['Count'] = dataset['User-ID'].map(users_counts)

In [24]:
dataset = filter_rows_by_values(dataset, "Count", list(range(200, max(dataset['Count']) + 1)))


In [25]:
# statistics on explicit dataset after removing users with more than 200 int.
print(f"New dataset statistics (users with interactions < {200}): ")
print(f"> No. of users: {len(dataset['User-ID'].unique())}")
print(f"> No. of Books: {len(dataset['ISBN'].unique())}")
print(f"> No. of Interaction: {dataset.shape[0]}")

New dataset statistics (users with interactions < 200): 
> No. of users: 77660
> No. of Books: 156891
> No. of Interaction: 364245


In [26]:
user_interaction, item_interaction = 1, 1

while user_interaction != 0 or item_interaction != 0:
    print(f"The current number of user and item with < 5 interactions: ")
    # user side fewer than ds_rate cheking
    uid_value_counts = dataset['User-ID'].value_counts()
    user_interaction = uid_value_counts[uid_value_counts < 5].count()
    print(f"No. of users < 5 ineractions: {user_interaction}")

    users_counts = dataset['User-ID'].value_counts()
    users_counts = users_counts.to_dict() #converts to dictionary
    dataset['Count'] = dataset['User-ID'].map(users_counts)

    dataset = filter_rows_by_values(dataset, "Count", list(range(5)))

    # item side fewer than ds_rate cheking
    bid_value_counts = dataset['ISBN'].value_counts()
    item_interaction = bid_value_counts[bid_value_counts < 5].count()
    print(f"No. of items < 5 ineractions: {item_interaction}")

    items_counts = dataset['ISBN'].value_counts()
    items_counts = items_counts.to_dict() #converts to dictionary
    dataset['Count'] = dataset['ISBN'].map(items_counts)

    dataset = filter_rows_by_values(dataset, "Count", list(range(5)))

The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 63585
No. of items < 5 ineractions: 119254
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 5825
No. of items < 5 ineractions: 1594
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 432
No. of items < 5 ineractions: 208
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 69
No. of items < 5 ineractions: 45
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 19
No. of items < 5 ineractions: 17
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 11
No. of items < 5 ineractions: 13
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 6
No. of items < 5 ineractions: 2
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 2
No. of items < 

In [27]:
# statistics on 5 rate explicit dataset (after pre-processing)
print(f"No. of users: {len(dataset['User-ID'].unique())}")
print(f"No. of Books: {len(dataset['ISBN'].unique())}")
print(f"No. of Interaction: {dataset.shape[0]}")

No. of users: 6358
No. of Books: 6921
No. of Interaction: 88552


In [28]:
# Before we save the preprocessed explicit dataset (5Rate) we first remove the added column which is `Count`
del dataset['Count']

In [30]:
dataset.to_csv("data/fairbook_ratings.csv")